In [ ]:
%pip install torch xgboost tensorflow keras

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import xgboost as xgb

from sklearn.metrics import accuracy_score, confusion_matrix, mean_absolute_error, mean_squared_error, r2_score, classification_report, f1_score, accuracy_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.utils.class_weight import compute_class_weight

from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.src.saving import load_model

pd.set_option('display.max_columns', None)

df = pd.read_csv('data.csv', on_bad_lines="warn", delimiter=";")
df.head()

As our hypothetical use case, we imagine a government agency that is interested to predict how much endangered fish a given catch operation has caught.

We used "https://artsdatabanken.no/lister/rodlisteforarter/2021?Name=&SortBy=ScientificName&Meta=Visited&Meta=scroll_658&IsCheck=Area&Area=N&IsCheck=Category&Category=CR&Category=EN&IsCheck=SpeciesGroups&SpeciesGroups=Fisker&IsCheck=Insekter" to find endangered species of fish. 

Three of these are included in the dataset:

1. Vanlig Uer
2. Blålange
3. Ål

Our strategy will be to trim unnecessary columns, turn species into boolean columns and have machine learning models predict the amount (weight) caught of the given species for each catch operation.

<!-- A lot of the data in the columns needs to be converted into boolean and number values, and some needs to be omitted due to inaccuracies.  -->




**Data Processing**

In [ ]:
# df.info()
uer_df = df[df['Art FAO'] == "Uer (vanlig)"] 
blålange_df = df[df['Art FAO'] == "Blålange"] 
ål_df = df[df['Art FAO'] == "Ål"] 
print("Number of catches containing Uer: ", len(uer_df))
print("Number of catches containing Blålange: ", len(blålange_df))
print("Number of catches containing Ål: ", len(ål_df))


There are only two catch operations containing Ål, we need more data in order to predict for this species.
For Blålange and Uer we can move on. It's difficult to say beforehand wether or not 991 positive datapoints will be enough to accurately predict the Rundvekt of Blålange, but it's within the realm of possibilities.
 

To begin trimming down the amount of data, we can safely drop columns where the data is already present in other columns

In [ ]:
df = df.drop(labels=["Meldingsdato","Meldingsklokkeslett","Startdato","Startklokkeslett", "Stoppdato", "Stoppklokkeslett", "Lengdegruppe (kode)", "Lengdegruppe"], axis=1)
df = df.drop(labels=["Hovedart FAO (kode)","Hovedart FAO","Hovedart - FDIR (kode)","Art - gruppe","Art - gruppe (kode)", "Art - FDIR (kode)", "Art - FDIR", "Redskap FDIR (kode)", "Redskap FDIR"], axis=1)



We notice that Start/Stop times are inconsistent. We therefore filter out Start/Stop times which do not contain ':'. This way we can trust the 'Varighet' column more.

Furthermore, 'Fartøylengde', 'Bredde', 'Startposisjon bredde', 'Startposisjon lengde', 'Stopposisjon bredde', 'Stopposisjon lengde' is written with ',' and not '.'. We change ',' to '.' and cast to float datatype.

In [ ]:
df = df[df["Starttidspunkt"].str.contains(":")]
df = df[df["Stopptidspunkt"].str.contains(":")]

castable_columns = ["Fartøylengde", "Bredde", "Startposisjon bredde", "Startposisjon lengde", "Stopposisjon bredde", "Stopposisjon lengde"]

for column in castable_columns:
    df[column] = df[column].str.replace(",", ".").astype("float")



Next we need to convert the categorical column values we _do_ want to use into machine-readable data. In this case we use the built-in get_dummies function to create boolean columns.

In [ ]:
species_columns = pd.get_dummies(df["Art FAO"], dtype="float") 
species_columns_multiplied = species_columns.multiply(df["Rundvekt"], axis="index") # Keep weight of fish

endangered_species_1 = species_columns_multiplied["Uer (vanlig)"]
endangered_species_2 = species_columns_multiplied["Blålange"]

# Add endangered species and drop original columns.
df = pd.concat([df,endangered_species_1 ], axis=1)
df = pd.concat([df,endangered_species_2 ], axis=1)
df = df.drop(["Art FAO", "Art FAO (kode)" ], axis=1)

tool_columns = pd.get_dummies(df["Redskap FAO"])
df = pd.concat([df,tool_columns], axis=1) 

df = df.drop(["Redskap FAO", "Redskap FAO (kode)"], axis=1)


In order to reduce the amount of parameters, we group the fishing tools into sensible categories.

In [ ]:
groups_to_combine = {
    'DivBunntrål': ['Bunntrål (uspes)', 'Bunntrål, otter', 'Bunntrål, par', 'Bunntrål, reke', 'Skrape, mekanisert', 'Otter dobbeltrål'],
    'DivSnurrevad': ['Snurrevad, skotsk', 'Snurrevad, dansk', 'Snurrevad'],
    'DivSnurpenot/ringnot': ['Snurpenot/ringnot, to fartøy', 'Snurpenot/ringnot, et fartøy', 'Snurpenot/ringnot', 'Boat /vessel seines -Pair seines'],
    'DivFlytetrål': ['Flytetrål, otter', 'Flytetrål, par', 'Flytetrål, reke', 'Flytetrål, uspesifisert'],
    'DivGarn': ['Udefinert garn', 'Settegarn', 'Gillnets and entangling nets (unspec)*', 'Encircling gillnets*'],
    'DivLine': ['Andre liner', 'Setteline', 'Dorg/harp/snik', 'Juksa/pilk, manuell'],
    'Annet': ['Harpun,div.', 'Partrål, uspesifisert', 'Teiner', 'Udefinert trål', 'Annen trål (udefinert)']
}

for new_column, columns_to_group in groups_to_combine.items():
    df[new_column] = df[columns_to_group].any(axis=1)
    df.drop(columns=columns_to_group, inplace=True)

df.head()

When grouping the different catch operations, it's important to be precise when telling pandas how to process the different rows.

The data in most columns is identical, for those we can use 'mean' or 'first'.

For some, like the endangered species columns, only one row will contain our fish, rest will be zero. We must therefore use 'sum' to capture the weight of the endangereed species.

For 'Rundvekt' we should also use 'sum'. 

In [ ]:
species_dict = {
    "Uer (vanlig)": "sum",
    "Blålange": "sum",
}
tool_dict= dict.fromkeys(['Annet','DivBunntrål','DivSnurrevad','DivSnurpenot/ringnot','DivFlytetrål','DivGarn','DivLine'], "first")

catch_dict = {
        "Varighet": "mean",
        "Fangstår": "mean",
        "Trekkavstand": "mean",
        "Bredde": "mean",
        "Fartøylengde" : "mean",
        "Bruttotonnasje": "mean",
        "Rundvekt": "sum",
        "Havdybde start": "mean",
        "Havdybde stopp": "mean",
        "Startposisjon lengde": "mean",
        "Startposisjon bredde": "mean",
        "Stopposisjon lengde": "mean",
        "Stopposisjon bredde": "mean",

    }
master_dict = {**catch_dict, **tool_dict, **species_dict}

It's natural to think that the size of the vessel is important for predicting how much fish is caught.

In order to use 'Bruttotonnasje' one option is to pick one of the columns, delete the other one and drop all n/a values.

We decided on a slightly different approach in order to preserve more rows.
We base the 'Bruttotonnasje' value on the standard 'Bruttotonnasje 1969' and instead of dropping rows right away, we check if the row contains a value in 'Bruttotonnasje annen', and use that.
Rows with missing values in both columns are dropped.


In [ ]:

df = df.dropna(subset=["Bruttotonnasje 1969", "Bruttotonnasje annen"], how='all')
df["Bruttotonnasje"] = df["Bruttotonnasje 1969"].fillna(df["Bruttotonnasje annen"])
df = df.drop(["Bruttotonnasje 1969", "Bruttotonnasje annen"], axis=1)


It should be noted that 'Bruttotonnasje' is pretty well explained by 'Fartøylengde', although in a non-linear fashion. In light of this, we could probably omit either 'Bruttotonnasje' or the boat dimensions.
See graphs.

In [ ]:

df.plot(kind = 'scatter', x = 'Fartøylengde', y = 'Bruttotonnasje', c="#43ff640d")
plt.show()

In [ ]:

df.plot(kind = 'scatter', x = 'Fartøylengde', y = 'Uer (vanlig)', c="#888")
plt.show()

In [ ]:

df.plot(kind = 'scatter', x = 'Fartøylengde', y = 'Blålange', c="#888")
plt.show()

Some catch operations are claimed to have taken place in very unusual and sometimes impossible locations.
Furthermore, some weights of the endangered species are incredibly high. We suspect this will distort the model, so we remove those.

In [ ]:
df = df.drop(df[df["Startposisjon lengde"] < -50].index)
df = df.drop(df[df["Startposisjon lengde"] > 60].index)
df = df.drop(df[df["Startposisjon bredde"] < 40].index)
df = df.drop(df[df["Uer (vanlig)"] > 25000].index) # Outlier
df = df.drop(df[df["Blålange"] > 8000].index) # Outlier


Geographic Visualization of where Uer is caught surrounding Norway. 

In [ ]:
# initialize an axis
fig, ax = plt.subplots(figsize=(8,6))
# plot map on axis
countries = gpd.read_file(  
     gpd.datasets.get_path("naturalearth_lowres"))
countries.plot(color="lightgrey",
                                                 ax=ax)
df_positive_uer = df.drop(df[df["Uer (vanlig)"] == 0].index)

# plot points
df_positive_uer.plot(x="Startposisjon lengde", y="Startposisjon bredde", kind="scatter", 
        c="Uer (vanlig)", colormap="YlOrRd", s=df_positive_uer["Uer (vanlig)"].multiply(0.003),
        ax=ax)
# add grid
ax.grid( alpha=0.5)
ax.set_xlim(0, 40)
ax.set_ylim(55, 80)
plt.show()


Geographic Visualization of where Blålange is caught surrounding Norway. 

In [ ]:
# initialize an axis
fig, ax = plt.subplots(figsize=(8,6))
# plot map on axis
countries = gpd.read_file(  
     gpd.datasets.get_path("naturalearth_lowres"))
countries.plot(color="lightgrey",
                                                 ax=ax)

df_positive_lange = df.drop(df[df["Blålange"] == 0].index)
# plot points
df_positive_lange.plot(x="Startposisjon lengde", y="Startposisjon bredde", kind="scatter", 
        c="Blålange", colormap="YlOrRd", s=df_positive_lange["Blålange"].multiply(0.04),
        ax=ax)
# add grid
ax.grid( alpha=0.5)
ax.set_xlim(-20, 35)
ax.set_ylim(50, 80)
plt.show()

We are now ready to collapse the entries into catch operations.

In [ ]:

df_processed = df.groupby(by=["Melding ID", "Starttidspunkt", "Stopptidspunkt"]).agg(master_dict).reset_index()
df_processed = df_processed.drop(columns=['Starttidspunkt','Stopptidspunkt',"Melding ID"])
df_processed = df_processed.dropna()

df_knn = df_processed.copy()
df_dl = df_processed.copy()
df_xg = df_processed.copy()
df_clustering = df_processed.copy()


***KNN Regressor model***


A k-nearest neighbors (KNN) regressor is a non-parametric machine learning model used for regression tasks. It predicts the value of a target variable by averaging the values of the k-nearest neighbors in the feature space.

Because of few observations of Blålange we focus on predicting Uer (vanlig).


In [ ]:
X = df_knn.drop('Uer (vanlig)', axis=1)
y = df_knn['Uer (vanlig)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

The code under performs a grid search to optimize the hyperparameters of a KNN regressor model using 5-fold cross-validation. It searches through a predefined set of possible values for the number of neighbors ("n_neighbors") and the weighting function ("weights") to find the combination that minimizes the negative mean squared error. This approach is used to systematically work through multiple combinations of parameter tunes, cross-validating as it goes to determine which tune gives the best performance.

In [ ]:

def getBestRegressor():
    # Defining parameter grid
    param_grid = {'n_neighbors': [3, 4, 5, 6, 7, 10, 15, 20], 'weights': ['uniform', 'distance']}
    grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train_scaled, y_train)
    # finding the best parameters
    print("Best parameters:", grid_search.best_params_)
    best_knn_regressor = grid_search.best_estimator_
    return best_knn_regressor



In [ ]:

# knn_regressor = getBestRegressor() # Function takes very long time to run.
knn_regressor = KNeighborsRegressor(n_neighbors=3, weights="distance")
knn_regressor.fit(X_train_scaled, y_train)
y_pred = knn_regressor.predict(X_test_scaled)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R^2 Score: {r2}")

MSE: Mean Squared Error is a measure of the average squared difference between the observed actual amount of Uer (vanlig) and the  predicted amount of Uer (vanlig) by the model, indicating a value of 122,397.15 in this case.

RMSE: Root Mean Squared Error is the square root of MSE, providing a measure of the average magnitude of the errors, with a value of 349.85 suggesting the typical prediction error's size.

MAE: Mean Absolute Error (MAE) is the average of the absolute differences between predicted values of Uer (vanlig) and actual values, with a value of 49.21 indicating the average prediction error in the same units as the data.

R^2 Score: The R-squared score, often called the coefficient of determination, is the proportion of the variance in the dependent variable that is predictable from the independent variables, with a value of 0.52 indicating that approximately 52.19% of the variability in the outcome is explained by the model.

We now want to visualize the predictions made by our model.


In [ ]:
def plotPredictions(test,pred):
    plt.figure(figsize=(10, 6))
    plt.scatter(test, pred, alpha=0.5)
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('Real values (log scale)')
    plt.ylabel('Predicted values (log scale)')
    plt.title('Real values vs. Predicted values for Uer (log scale)')

    plt.plot([min(test), max(test)], [min(test), max(test)], 'k--')
    plt.show()

plotPredictions(y_test, y_pred)

The log graph indicates that the model is capable of predicting the correct values to a certain extent, as evidenced by the clustering of data points along the line of perfect prediction, although there is still some deviation present.

In [ ]:
residuals = y_test - y_pred
residuals = residuals[~np.isnan(residuals)]
residuals = residuals[np.isfinite(residuals)]

# Distribution of 10% random sample of the data resudials
test_size = int(len(residuals) * 0.1)
random_sample = np.random.choice(residuals, size=test_size, replace=False)

plt.xlim(-800, 1500)
sns.histplot(random_sample, kde=True)
plt.title('Distribution of the residuals (10% random samle)')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()

The residuals being centered around zero suggests that the model's predictions are, on average, accurate, as there is no systematic overprediction or underprediction. This indicates a good fit between the model and the observed data.

***Tensorflow deep learning model***

A TensorFlow deep learning model is a computational architecture composed of multiple layers of interconnected nodes, or "neurons," designed to recognize patterns in data by adjusting internal parameters through a process known as training. TensorFlow is an open-source machine learning library developed by Google that facilitates the creation, training, and deployment of such models for a wide variety of complex tasks, including image and speech recognition, natural language processing, and more.


In [ ]:


min_val = df_dl['Uer (vanlig)'].min() # null
max_val = df_dl['Uer (vanlig)'].max()
bands = [min_val, 10, 100 , max_val]

print(min_verdi)
print(max_verdi)
kategorier = ['few', 'some', 'many']
df_dl['Uer_kategorisert'] = pd.cut(df_dl['Uer (vanlig)'], bins=bands , labels=kategorier, include_lowest=True)

y = df_dl['Uer_kategorisert']
y = y.astype('category').cat.codes

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Too few non-zero observations, need to weigh the observations we do have.
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_encoded), y=y_encoded)
class_weights_dict = dict(enumerate(class_weights))

X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

model = Sequential([
    Input(shape=(X_train.shape[1],)),  
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train_encoded, epochs=10, class_weight=class_weights_dict, validation_data=(X_test, y_test_encoded))




In [ ]:

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

conf_mat = confusion_matrix(y_test_encoded, y_pred_classes) 
print("Forvirringsmatrise: \n", conf_mat)

class_report = classification_report(y_test_encoded, y_pred_classes, target_names=['few', 'some', 'many'])  
print("\nKlassifiseringsrapport: \n", class_report)


***XGBoost Algo***


XGBoost, short for eXtreme Gradient Boosting, is an ensemble machine learning algorithm known for its efficiency and scalability in training models. It enhances accuracy by combining the predictions of several simpler models into a powerful composite predictor. 

The algorithm has a track record of winning numerous machine learning contests, making it not only a sound choice for the project but also a trendy one. One of XGBoost's key features is its efficient handling of missing data, which allows it to manage real-world datasets with missing values without great preprocessing. 

Incorporating XGBoost as one of our supervised learning models aligns with the precision required for our hypothetical use case and adds a touch of trendiness and spiciness to the project.


In [ ]:

df_xgboost = df_xgboost.drop("Blålange", axis = 1)

# Defining the independent variables (X) and the dependent varaible (y).
y = df_xgboost['Uer (vanlig)']
X = df_xgboost.drop('Uer (vanlig)', axis=1)

# Splitting the dataset into training and test set with a 70-30 split.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

df_xgboost


Model Tuning and Hyperparamterer Optimalization with Grid Search:

After splitting the dataset, it is important to perform fine-tuning on our model's hyperparameters. Tuning can ensure that we enhance the model's performance. We use Grid Search for this purpose, which methodically explores a range of possibilities for hyperparameters to find the best combinations. This process is computationally demanding, but it is crucial to avoid overlooking more optimized settings for our task.

In [ ]:

def calculateOptimizedParameters():
    # Defining a reasonable paramterer grid or distribution to sample from.
    param_grid = {
        'colsample_bytree': [0.3, 0.5, 0.7],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'alpha': [1, 10, 50],
        'n_estimators': [200, 800, 1500]
    }

    xg_reg = xgb.XGBRegressor(objective='reg:squarederror')
    grid_search = GridSearchCV(estimator=xg_reg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)
    grid_search.fit(X_train, y_train)

    print(f"Best parameters found: {grid_search.best_params_}")

# calculateOptimizedParameters()


Result of Grid Search

The Grid search is a computationally demandning process and it can be time-consuming.
On my computer, the search took about 15 minutes. Therefore, I have included the results here.
The optimal hyperparamters identified by Grid Search for our XGBoost model are as follows:

alpha: 10

colsample_bytree: 0.5

learning_rate: 0.1

max_depth: 7

n_estimators: 200



In [ ]:
# Initializing the the XGBoost regressor with hyperparamters.
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                        colsample_bytree = 0.7, 
                        learning_rate = 0.1,
                        max_depth = 7, 
                        alpha = 10, 
                        n_estimators =800)


xg_reg.fit(X_train, y_train)

y_pred = xg_reg.predict(X_test)

# Evaluering av modellen

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE (Mean Squared Error): {mse}")
print(f"RMSE (Root Mean Squared Error): {rmse}")
print(f"MAE (Mean Absolute Error): {mae}")
print(f"R^2 Score: {r2}")


Feature Analysis

Before we examine the model's predictions in detail, it is important to understand the influence of different features on the model's output. Feature analysis helps us identify which features significantly impact the predictions and provides insights into the relationships within our data.



In [ ]:
xgb.plot_importance(xg_reg)
plt.show()

Visualizing the Predictions

After examining the importance of different features, we will now provide a visualization of the model's predictions. By plotting actual values against predicted values, we can visualize the model's accuracy. The plot helps us visualize how closely the model's predictions align with the ideal representation of perfect agreement between predicted and actual values.


In [ ]:

plotPredictions(y_test, y_pred)


Residual Analysis

Residual analysis is used in evaluation a regression models performance. The analysis examines the difference between actual and predicted values. Randomly distributed residuals tend to suggest a well-fitting model. 



In [ ]:
residuals = y_test - y_pred
plt.scatter(y_test, residuals)
plt.hlines(y=0, xmin=y_test.min(), xmax=y_test.max(), colors='r', linestyles='--')
plt.xlabel('Faktiske verdier')
plt.ylabel('Residualer')
plt.title('Residual Plot')
plt.show()








***K-Means Clustering***

is an unsupervised machine learning algorithm used to identify and group similar data points into clusters. The algorithm partitions the data points into K distinct clusters, aiming to minimize the variance within each cluster and maximize the variance between clusters. We use K-Means to detect patterns that might be hidden within our data, without the need to predefine labels.





In [ ]:
relevant_columns = ["Startposisjon bredde", "Startposisjon lengde", 
    "Havdybde start", "Varighet", "Fangstår", 
    "Stopposisjon bredde", "Stopposisjon lengde", 
    "Havdybde stopp", "Trekkavstand", "Rundvekt", 
    "Bredde", "Fartøylengde", "Bruttotonnasje"]

df_clustering = df_clustering[relevant_columns]
scaler = StandardScaler()
df_clustering_scaled = scaler.fit_transform(df_clustering)


In [ ]:
df_clustering.head()

***The Elbow Method***

To effectively utilize K-Means clustering, it is important to first determine an appropriate number of clusters. We have chosen the Elbow Method to find this number by calculating the inertia for different values of K.

Inertia measures the sum of squared distances of samples to their nearest cluster center. By plotting inertia values for different values of K, we can identify what is known as the "Elbow Point". At this point, the rate of decrease in the graph sharply changes.

This point suggests a balance between the number of clusters and the compactness of the clusters, which in turn indicates a suitable value for K in K-Means.


In [ ]:
# Implementing the Elbow Method. 

inertia = []

# Calculating the intertia for a range of clusters (k).
for k in range(1, 21):
    kmeans = KMeans(n_clusters=k, random_state=42) 
    kmeans.fit(df_clustering_scaled)
    inertia.append(kmeans.inertia_)

plt.plot(range(1, 21), inertia)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

After plotting the graph, we find that a suitable value for K in K-means is K = 6.

In [ ]:
k = 6

kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(df_clustering_scaled)

labels = kmeans.labels_

df_clustering_2 = df_processed.copy()
df_clustering_2["cluster"] = labels

for i in range(k):
    cluster_data = df_clustering_2[df_clustering_2["cluster"] == i]
    print(f"Cluster {i}:")
    # Sørg for at kolonnene 'Uer (vanlig)' og 'Blålange' eksisterer i df_processed før du prøver å få tilgang til dem
    if 'Uer (vanlig)' in df_clustering_2.columns and 'Blålange' in df_clustering_2.columns:
        print(f"Vanlig Uer: {cluster_data['Uer (vanlig)'].mean()}")
        print(f"Blålange: {cluster_data['Blålange'].mean()}")
    else:
        print("En eller begge kolonnene 'Uer (vanlig)' og 'Blålange' finnes ikke i DataFrame.")

***Principle Component Analysis***

We utilize a technique known as Prinicple Component Analysis (PCA) to reduce the dimensionality of the dataset by transforming into principal components. These orthogonal, uncorrelated components captures the greatest variance in the dataset, across fewer dimensions. 

PCA enables us to project the data into a lower-dimensional plot to visualize the clusters. In our task, we have used PCA to facilitate a 2D plot, which allows for the interpretation of cluster groupings and visualize separations that might be obscured in higher-dimensional spaces.



In [ ]:

# Reducing to 2 dimensions and transforming the scaled dataset to find the components that best captures the variance.
pca = PCA(n_components=2)  
principal_components = pca.fit_transform(df_clustering_scaled)

# Creating a dataframe with the components to plot it in a 2D space.
df_visual = pd.DataFrame(data=principal_components, columns=["PC1", "PC2"])
df_visual["cluster"] = labels  

# Creating a scatter plot to visualize the clusters.
plt.figure(figsize=(10, 10))
for i in range(k):
    cluster_subset = df_visual[df_visual['cluster'] == i]
    plt.scatter(cluster_subset['PC1'], cluster_subset['PC2'], label=f'Cluster {i}')

plt.title('2D PCA Clustering Visualization')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.show()

***Analyzing Cluster Centroid Properties***

Analyzing the properties of cluster centroids allows us to gain important insights into unique clusters characteristics and defining traits. 
This understanding can reveal underlying patterns within our dataset, providing context for the cluster behaviour. 



In [415]:
# Gathering the cluster centers identifed by K-means and creating a dataframe.
centroids = kmeans.cluster_centers_
centroids_df = pd.DataFrame(scaler.inverse_transform(centroids), columns=relevant_columns)

# Printing the centroids of each cluster.
for i in range(k):
    print(f"Sentroid for klynge {i}:")
    print(centroids_df.loc[i])
    print()

# Analyzing and printing important information about clusters.
for i in range(k):
    cluster_data = df_processed_dropna[df_processed_dropna["cluster"] == i]
    print(f"Cluster {i}:")
    if 'Uer (vanlig)' in df_processed_dropna.columns and 'Blålange' in df_processed_dropna.columns:
        print(f"Vanlig Uer: {cluster_data['Uer (vanlig)'].mean()}")
        print(f"Blålange: {cluster_data['Blålange'].mean()}")
    else:
        print("Blålange and Uer does not exists in one or both columns of the dataframe")



Sentroid for klynge 0:
Startposisjon bredde       59.874907
Startposisjon lengde        4.462447
Havdybde start           -188.482127
Varighet                  611.972271
Fangstår                 2018.000000
Stopposisjon bredde        59.870554
Stopposisjon lengde         4.510221
Havdybde stopp           -188.394671
Trekkavstand            18072.352289
Rundvekt                 8485.358451
Bredde                      7.914817
Fartøylengde               30.136005
Bruttotonnasje            345.719674
Name: 0, dtype: float64

Sentroid for klynge 1:
Startposisjon bredde      71.034736
Startposisjon lengde      22.385406
Havdybde start          -183.021476
Varighet                 321.335265
Fangstår                2018.000000
Stopposisjon bredde       71.036953
Stopposisjon lengde       22.415618
Havdybde stopp          -181.138486
Trekkavstand            5747.265326
Rundvekt                9273.232316
Bredde                     7.921340
Fartøylengde              30.104365
Bruttotonnasje  

***Tabular Comparison of Clusters***

Creating a tabular comparison to better visualize, simplify the data, to facilitate a more intuitive analysis in order to draw meaningful conclusions from our clustering.



In [416]:
# Calculating mean of the dataset as refernce.
overall_means = df_clustering.mean()

# Converting centroids to dataframe for analyzing purposes in tabular form.
centroids_df = pd.DataFrame(scaler.inverse_transform(centroids), columns=relevant_columns)

# Adding mean values of the endangered species.
cluster_means = []
for i in range(k):
    cluster_data = df_processed_dropna[df_processed_dropna["cluster"] == i]
    cluster_means.append({
        'Vanlig Uer': cluster_data['Uer (vanlig)'].mean(),
        'Blålange': cluster_data['Blålange'].mean()
    })

# Converting cluster means to a dataframe.
cluster_means_df = pd.DataFrame(cluster_means)

# Combinging the dataframes.
comparison_df = pd.concat([centroids_df, cluster_means_df], axis=1)

# Adding the mean values.
comparison_df.loc['Overall Mean'] = overall_means.tolist() + [df_processed_dropna['Uer (vanlig)'].mean(), df_processed_dropna['Blålange'].mean()]

# Printing the tables.
print(comparison_df)



              Startposisjon bredde  Startposisjon lengde  Havdybde start  \
0                        59.874907              4.462447     -188.482127   
1                        71.034736             22.385406     -183.021476   
2                        74.013188             28.195496     -249.782927   
3                        61.396602             -4.693854    -2186.346980   
4                        74.902250             15.939250     -381.750000   
5                        61.576475              0.678918     -234.116523   
Overall Mean             67.774773             16.279589     -252.924540   

                 Varighet     Fangstår  Stopposisjon bredde  \
0              611.972271  2018.000000            59.870554   
1              321.335265  2018.000000            71.036953   
2             1627.220359  2018.000000            74.026483   
3              266.137793  2018.000000            61.188758   
4              280.000000  2017.000000            74.916250   
5            

### What Did We learn from Clustering?

Based on the results from the clustering model, several observations can be made regarding the variability between the clusters, geographical distribution, catch operating vessel characteristics, duratio and wather depth.

***Variability In Clusters:***

Clusters 2, 3 and 4 predominantly report catches of Uer, with a negligable amount of Blålange.
Clusters 1 and 2 contraints both species. However, cluster 0 has approximately three times more Uer than Blålange, while cluster 0 has a more baanced amount of both.
Cluster 4 and 5 recorded almost no catches of both endangered species. Cluster 4 has an average of 9 caught Uer, and barely any Blålange, while cluster 5 has minimal catches of both.

***Geographical Insights:***

Clusters 0, 1 and 4 are distinct from clusters 2, 3 and 5. The separation in geographical areas may contribute to differences in species composition, suggesting that some areas may have a higher presence of Uer and Blålange.

***Catch Operation Duration:***

Most clusters exhibit similar durations for catch operations, with the exception of cluster 2, which has a significantly longer duration. This cluster also has a higher mean catch of Vanlig Uer, which is substantially greater than the second-highest mean catch recorded by cluster 0. This indicates that a longer catch operation, may contribute to catching more endangered species.

***Water Depth:***

Cluster 5 operates at a significantly lower water depth compared to the other clusters and the overall mean, almost 11 times deeper. The negligible catch of Blålange or Uer in this cluster may indicate that these species do not thrive at such deep waters.

***Vessel Characteristics:***

Vessel length, vessel width, whole fish weight, and gross tonnage reveals that clusters 1 and 3 have lower and similar values compared to the other clusters and the overall means. These clusters reports low catches of endangered species, which might suggest that smaller vessels, catch less fish, and consequently, less endangered fish. 


**Results and comments**

TODO: "write a summary of your results, and discuss consequences of such results."
